<a href="https://colab.research.google.com/github/motahari64/MedBot---Medical-Chatbot/blob/main/Medbot_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain openai tiktoken unstructured chromadb

In [ ]:
import os
#Specify your API key in OPENAI
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/')



**Build a vectorDB**

In [ ]:
# Build a sample vectorDB
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

#Specify the path to the folder containing the TXT files
path = ''
loader = DirectoryLoader(path, glob="**/*.txt")
data=loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
len(data)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
## Now we need to create a tool for our retriever. The main things we need to pass in are a name for the retriever as well as a description.
## These will both be used by the language model, so they should be informative.

from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    retriever,
   "search_documents",
    "Searches and returns documents regarding the txt file in resource folder",
)

tools = [tool]

In [ ]:
question_input = input("what is your question ? ")

In [ ]:
from langchain.chains import create_extraction_chain
from langchain.chat_models import ChatOpenAI

# Schema
schema = {
    "properties": {
        "sickness": {"type": "string"},
        "drug": {"type": "string"},
        "symptoms": {"type": "string"},
        "symptoms_organ": {"type": "string"},
        "symptoms_extra_info": {"type": "string"}
    },
    "required": ["sickness","drug","symptoms"],
}

# Input
inp = question_input
# Run chain
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)

extract=chain.run(inp)


chain.run(inp)         ##show all



**Agent Constructor**

In [ ]:
## Here, we will use the high level create_conversational_retrieval_agent API to construct the agent.

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [ ]:
from langchain.prompts import PromptTemplate



prompt_template = PromptTemplate.from_template(
    "answer the question this : {question} by search only available resource text. Extract the answer to the question '{question}' or say 'not found' if the information is not contained."
)
if(extract):
  agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)
  result = agent_executor(
    {
        "input":prompt_template.format(question=question_input)
    }
  )
else : print("I am medical assistant, please ask your medical question only")

result["output"]
